In [2]:
import torch
from sympy import Matrix, init_printing, cos, sin, rad, N
import random
import time
from concurrent.futures import ProcessPoolExecutor
import timeit
import subprocess

# Initialize pretty printing
init_printing()


In [2]:
print(torch.__version__)

2.3.0.dev20240311


$$
\mathbf{A} = \begin{pmatrix} {11} & {12} & {13} \\ {21} & {22} & {23} \end{pmatrix}
$$


$$

a \in \mathbb{R}
$$

$$

\mathbf{M} = \begin{pmatrix} 
m_{11} & m_{12} & \cdots & m_{1n} \\ 
m_{21} & m_{22} & \cdots & m_{2n} \\ 
\vdots & \vdots & \ddots & \vdots \\ 
m_{m1} & m_{m2} & \cdots & m_{mn} 
\end{pmatrix}
$$
$$
\mathbf{M}^\top
$$

$$
\mathbf{M}^{-1}
$$

$$
\vec{u} \cdot \vec{v} = u_1v_1 + u_2v_2 + \cdots + u_nv_n

$$

$$
\mathbf{T} = T_{ijk} \quad \text{where } i, j, k \text{ are indices.}
$$

In [7]:

x = torch.rand(5, 3)
print(x)

tensor([[0.1670, 0.3071, 0.7835],
        [0.1887, 0.1755, 0.5571],
        [0.8769, 0.2957, 0.8682],
        [0.4411, 0.0449, 0.9226],
        [0.6492, 0.6468, 0.1082]])


$$
\mathbf{A} = \begin{pmatrix}
a & b \\
c & d
\end{pmatrix}
\\
\mathbf{A}^{-1} = \begin{pmatrix}
d & -b \\
-c & a
\end{pmatrix}
$$

$$
\mathbf{AA}^{-1} = \begin{pmatrix}
ad & -bc \\
-cb & ad
\end{pmatrix}
$$

In [30]:
# Define your vector
vector = [1, 2, 3]

# Convert the vector to a LaTeX formatted string
Matrix(vector)


⎡1⎤
⎢ ⎥
⎢2⎥
⎢ ⎥
⎣3⎦

In [3]:
len1 = 8
deg1 = 190
len2 = 3
deg2 = 310

# vector = [
#     N(cos(rad(deg1))*len1 + cos(rad(deg2))*len2), 
#     N(sin(rad(deg1))*len1 + sin(rad(deg2))*len2)]
# vector



In [3]:
def vec(len, deg):
    return len*Matrix([cos(rad(deg)), sin(rad(deg))])

v = vec(8, 190)+vec(3, 310)
N(v)

⎡-5.95009919503805⎤
⎢                 ⎥
⎣-3.68731875069238⎦

In [4]:
def det(m):
    size = len(m)
    det = 1
    for row in range(size):
        swap = row+1
        while m[row][row] == 0:
            if (swap == size):
                return 0
            m[row], m[swap] = m[swap], m[row]
            det*=-1
            swap+=1
        for op in range(row+1, size):
            for col in range(row+1, size):
                m[op][col] = m[op][col] - m[row][col] * m[op][row] / m[row][row]
            m[op][row] = 0
    for i in range(size):
        det*=m[i][i]
    return det

def mult(m1, m2):
    dim = len(m1[0])
    if dim != len(m2):
        raise ValueError("Invalid matrix dimensions")
    rows = len(m1)
    cols = len(m2[0])
    result = [[0 for c in range(cols)] for r in range(rows)]
    for r in range(rows):
        for c in range(cols):
            for d in range(dim):
                result[r][c] += m1[r][d]*m2[d][c]
    return result
    
def rand_matrix(size):
    return [[torch.rand(1).item() for i in range(size)] for j in range(size)]

In [5]:
vec = rand_matrix(10)
ten = torch.tensor(vec).float()
m = Matrix(vec)

cycles = 1000

start = time.time()
a = det(vec)
end = time.time()
print("mine: " + str((end-start)*1000000))

start = time.time()
b = torch.det(ten)
end = time.time()
print("torch: " + str((end-start)*1000000))


print("mine: " + str(a))
print("torch: " + str(b.item()))
N(m)

mine: 92.02957153320312
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
torch: 108270.16830444336
mine: -0.011910630611473502
torch: -0.01191063690930605


⎡0.991800606250763    0.538183689117432   0.284696042537689   0.20332401990890
⎢                                                                             
⎢0.336105108261108    0.593875408172607   0.0190650224685669  0.92434370517730
⎢                                                                             
⎢0.446864366531372    0.143027901649475   0.761741101741791   0.95378446578979
⎢                                                                             
⎢0.193450331687927    0.828258454799652   0.457113564014435   0.73031246662139
⎢                                                                             
⎢0.910289227962494    0.854545891284943   0.463100790977478   0.13410645723342
⎢                                                                             
⎢0.392953634262085    0.871023178100586    0.09908527135849   0.72408950328826
⎢                                                                             
⎢0.849647343158722    0.95172917842865     0.6946836

In [ ]:




# ts1 = []
# for i in range(cycles):
#     ts1.append(torch.tensor(rand_matrix(9)).float())

# ts2 = []
# for i in range(cycles):
#     ts2.append(torch.tensor(rand_matrix(9)).float())

# start = time.time()
# for i in range(1):
#     m1 = mult(v1, v2)
# end = time.time()
# print("mine: " + str((end-start)*1000000))

# start = time.time()
# for i in range(cycles):
#     m2 = ts1[i] @ ts2[i]
# end = time.time()
# print("torch: " + str((end-start)*1000000))



In [19]:
size = 2048

v1 = rand_matrix(size)
c2 = torch.tensor(v1).float()
g1 = torch.tensor(v1).float().to("mps")

v2 = rand_matrix(size)
c2 = torch.tensor(v2).float()
g2 = torch.tensor(v2).float().to("mps")

# %timeit t1 @ t2
# %timeit mult(v1, v2)

# Warm-up
# for _ in range(100):
#     torch.matmul(torch.rand(500,500).to("mps"), torch.rand(500,500).to("mps"))

start = time.time()
c2 @ c2
end = time.time()
print("cpu: " + str((end-start)*1000000))

start = time.time()
g2 @ g2
end = time.time()
print("gpu: " + str((end-start)*1000000))

# start = time.time()
# Matrix(mult(v1, v2))
# end = time.time()
# print("mine: " + str((end-start)*1000000))




cpu: 31103.849411010742
gpu: 483.7512969970703


In [ ]:
# Assuming your compiled C program is named 'matmul' and is in the current directory
start = time.time()
result = subprocess.run(['../c/matmul'])
end = time.time()
print("c: " + str((end-start)*1000000))

c: 1141057.9681396484
